In [18]:
import requests
import pandas as pd
import os
import matplotlib.pyplot as plt

# Set the API key
HEADERS = {
        "Accept": "application/json",
        "Authorization": "fsq3sBRRp8FffG0vdAxZKDQyxmSiK9QoQUhBBYoVNItiqjw=" 
    }

def extract_foursquare_data(df, headers=HEADERS):
    """
    Extracts data from the Foursquare API using the provided DataFrame and headers. The DataFrame should
    contain 'latitude' and 'longitude' columns. The function returns a new DataFrame with the extracted 
    data.

    :param df: DataFrame containing 'latitude' and 'longitude' columns
    :param headers: API headers for the Foursquare API
    :returns: DataFrame with the extracted Foursquare API data
    """

    foursquare_data_extract = []

    # Loop through the DataFrame rows
    for _, row in df[:10].iterrows():
        lat, lng = row['latitude'], row['longitude']
        url = f"https://api.foursquare.com/v3/places/nearby?ll={lat},{lng}"

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
        except requests.exceptions.RequestException as err:
            print(f"An error occurred: {err}")
            continue

        foursquare_data = response.json()

        for result in foursquare_data['results']:
            extracted_data = {
                "fsq_id" : result['fsq_id'],
                "category_id" : result['categories'][0]['id'],
                "category_name" : result['categories'][0]['name'],
                "chains" : result['chains'],
                "distance" : result['distance'],
                "latitude" : result['geocodes']['main']['latitude'],
                "longitude" : result['geocodes']['main']['longitude'],  
                "location_country" : result.get('location', {}).get('country'),
                "location_cross_street" : result.get('location', {}).get('cross_street'),
                "location_formatted_address" : result['location']['formatted_address'],  
                "location_locality" : result.get('location', {}).get('locality'),
                "location_postcode" : result.get('location', {}).get('postcode'),  
                "location_region" : result.get('location', {}).get('region'),   
                "location_timezone" : result['timezone'],   
                "name" : result['name']
            }
            foursquare_data_extract.append(extracted_data)

    # Convert the list of dictionaries to a DataFrame
    df2 = pd.DataFrame(foursquare_data_extract)

    return df2


In [19]:
foursquare_df.to_csv('G:/Data/Python Project/foursquare.csv') 

In [21]:
print(foursquare_df)

                      fsq_id  category_id      category_name  \
0   4b65b9f1f964a520fefb2ae3        13031       Burger Joint   
1   4d0cef321f6bf04d75707a31        16037         Playground   
2   4deb8ba688774880e3387c0c        17065     Farmers Market   
3   4fdbc663e4b029940a58c201        16049          Surf Spot   
4   4be3345b660ec928226dcb3b        16017             Garden   
..                       ...          ...                ...   
86  570ff104498eeab32a85da3e        18021     Gym and Studio   
87  618ca101ab4b8264f40b147a        13148  French Restaurant   
88  4c8bc4cf5e38b1f773f22af9        11073                Spa   
89  4b04191df964a5205c5122e3        11045               Bank   
90  56d5a00fcd109c07f4c4afa5        13034               Café   

                                               chains  distance   latitude  \
0   [{'id': 'ab4a3530-d68a-012e-5619-003048cad9da'...        62  43.665065   
1                                                  []        63  43.664679 

In [20]:
import os
import requests
import json
import pandas as pd

business_id = 'srJ-hGeN_7H5oGT4wYz7uw'
api_key = "w7GSO5abCgHWgnZ-Vd21P00zNIY_nx-1ygAPL-5XngiWsGeBAPuTaMVPSB4ulo3veq0cjeEiPtM9IzErrIWwwrI3Wt5ELsCrx5qT4xDp6UgHBe83PU7LebN4MRx-ZHYx"
headers = {'Authorization': 'Bearer %s' % api_key}

# Define the endpoint URL for bike stations
endpoint = 'https://api.yelp.com/v3/businesses/search'
params = {'term': 'Bike stations', 'location': 'Toronto', 'radius': 1000}

# Send GET request to Yelp API for bike stations
response = requests.get(endpoint, params=params, headers=headers)

# Check if the status code is 200 (OK)
if response.status_code == 200:
    print('The status code is 200')
    bike_yelp_details = json.loads(response.text)

    # Use json_normalize() to convert JSON to DataFrame
    df_bike_stations = pd.json_normalize(bike_yelp_details['businesses'])
    print(df_bike_stations)
else:
    print('Request failed with status code', response.status_code)


# Define the endpoint URL for restaurants
endpoint = 'https://api.yelp.com/v3/businesses/search'
params = {'term': 'Restaurants', 'location': 'Toronto'}

# Send GET request to Yelp API for restaurants
response = requests.get(endpoint, params=params, headers=headers)

# Check if the status code is 200 (OK)
if response.status_code == 200:
    print('The status code is 200')
    restuarnt_details = json.loads(response.text)

    # Use json_normalize() to convert JSON to DataFrame
    df_restaurants = pd.json_normalize(restuarnt_details['businesses'])
    restaurant_data = df_restaurants[['name', 'review_count', 'location.address1', 'rating']]
    print(restaurant_data)
else:
    print('Request failed with status code', response.status_code)

# Sort the restaurant data by rating in descending order
sorted_restaurant_data = restaurant_data.sort_values(by='rating', ascending=False)
print(sorted_restaurant_data.head(10))



The status code is 200
Empty DataFrame
Columns: []
Index: []
The status code is 200
                            name  review_count         location.address1  \
0      Pai Northern Thai Kitchen          3398          18 Duncan Street   
1                           Mira           263  420A Wellington Street W   
2               Richmond Station           887       1 Richmond Street W   
3                           Rasa           321         196 Robert Street   
4                        Katsuya           488       66 Gerrard Street E   
5                        Aloette           287        163 Spadina Avenue   
6                      Gusto 101          1122       101 Portland Street   
7              Trattoria Nervosa           390       75 Yorkville Avenue   
8                     Fieramosca           110  36A Prince Arthur Avenue   
9                          Irene             1         25 Dockside Drive   
10                        Byblos           898          11 Duncan Street   
11  